# Introduction to Gaussian Naive Bayes

Gaussian Naive Bayes is a probabilistic classification algorithm based on Bayes' Theorem, assuming that features follow a normal (Gaussian) distribution and are conditionally independent given the class label.

## Bayes' Theorem

$$
P(y \mid X) = \frac{P(X \mid y) \cdot P(y)}{P(X)}
$$

where:
- $P(y \mid X)$ is the posterior probability of class $y$ given features $X$.
- $P(X \mid y)$ is the likelihood of features $X$ given class $y$.
- $P(y)$ is the prior probability of class $y$.
- $P(X)$ is the probability of features $X$.

## Gaussian Likelihood

For each feature $x_i$, the likelihood under the Gaussian assumption is:

$$
P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma_{y,i}^2}} \exp\left(-\frac{(x_i - \mu_{y,i})^2}{2\sigma_{y,i}^2}\right)
$$

where:
- $\mu_{y,i}$ is the mean of feature $i$ for class $y$.
- $\sigma_{y,i}^2$ is the variance of feature $i$ for class $y$.

## Classification Rule

The predicted class $\hat{y}$ for a sample $X$ is:

$$
\hat{y} = \arg\max_{y} \; P(y) \prod_{i=1}^{n} P(x_i \mid y)
$$

Gaussian Naive Bayes is simple, efficient, and often effective for high-dimensional datasets.